In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Se conecta con la BD MySQL AWS

In [2]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Genera un df con el archivo `csv` "D_ICD_DIAGNOSES"

In [3]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\D_ICD_DIAGNOSES.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,icd9_code,short_title,long_title
0,1,01716,Erythem nod tb-oth test,Erythema nodosum with hypersensitivity reactio...
1,2,01720,TB periph lymph-unspec,"Tuberculosis of peripheral lymph nodes, unspec..."
2,3,01721,TB periph lymph-no exam,"Tuberculosis of peripheral lymph nodes, bacter..."


In [4]:
df = df.replace(np.nan, 0)

### Inserta los registros del df "D_ICD_DIAGNOSES" en la BD

In [5]:
sql_insert = ''' INSERT 
                   INTO  D_ICD_DIAGNOSES
                         (row_id, icd9_code, short_title, long_title)
                 VALUES  (%s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    icd9 = df.loc[i, 'icd9_code']
    shtl = df.loc[i, 'short_title']
    lgtl = df.loc[i, 'long_title']
    reg_s = (row, icd9, shtl, lgtl)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

### Confirma la inserción de los registros a la BD

In [6]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [7]:
cursor.close()
con.close()